[Link](https://runawayhorse001.github.io/LearningApacheSpark/regression.html)

#  Set up spark context and SparkSession


In [1]:
try:
    sc.stop
except:
    pass

from pyspark import SparkContext , SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sparkContext = sc)

#  Load dataset


In [2]:
df = spark.read.format('com.databricks.spark.csv').\
                       options(header='true', \
                       inferschema='true').\
            load("../data/Advertising.csv",header=True);
df.show(5,True)
df.printSchema()
df.describe().show()


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|             

#  Convert the data to dense vector (features and label)

In [3]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# I provide two ways to build the features and labels

# method 1 (good for small feature):
#def transData(row):
#    return Row(label=row["Sales"],
#               features=Vectors.dense([row["TV"],
#                                       row["Radio"],
#                                       row["Newspaper"]]))

# Method 2 (good for large features):
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

#  Transform the dataset to DataFrame


In [4]:
transformed= transData(df)
transformed.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



#  Deal With Categorical Variables


In [5]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5,True)


+-----------------+-----+-----------------+
|         features|label|  indexedFeatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



#  Split the data into training and test sets (40% held out for testing)

In [6]:
# Split the data into training and test sets (40% held out for testing)
(trainingData, testData) = transformed.randomSplit([0.6, 0.4])
trainingData.show(5)
testData.show(5)

+----------------+-----+
|        features|label|
+----------------+-----+
|  [0.7,39.6,8.7]|  1.6|
|  [4.1,11.6,5.7]|  3.2|
| [7.3,28.1,41.4]|  5.5|
| [7.8,38.9,50.6]|  6.6|
|[11.7,36.9,45.2]|  7.3|
+----------------+-----+
only showing top 5 rows

+----------------+-----+
|        features|label|
+----------------+-----+
|  [5.4,29.9,9.4]|  5.3|
|  [8.4,27.2,2.1]|  5.7|
|   [8.6,2.1,1.0]|  4.8|
| [8.7,48.9,75.0]|  7.2|
|[13.2,15.9,49.6]|  5.6|
+----------------+-----+
only showing top 5 rows



# RandomForestRegressor

In [7]:
# Import LinearRegression class
from pyspark.ml.regression import RandomForestRegressor

# Define LinearRegression algorithm
rf = RandomForestRegressor() # featuresCol="indexedFeatures",numTrees=2, maxDepth=2, seed=42

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])
model = pipeline.fit(trainingData)

#  Make predictions


In [9]:
# Make predictions.
predictions = model.transform(testData)
predictions

DataFrame[features: vector, label: double, indexedFeatures: vector, prediction: double]

In [10]:
predictions.select("features","label","indexedFeatures").show()

+----------------+-----+----------------+
|        features|label| indexedFeatures|
+----------------+-----+----------------+
|  [5.4,29.9,9.4]|  5.3|  [5.4,29.9,9.4]|
|  [8.4,27.2,2.1]|  5.7|  [8.4,27.2,2.1]|
|   [8.6,2.1,1.0]|  4.8|   [8.6,2.1,1.0]|
| [8.7,48.9,75.0]|  7.2| [8.7,48.9,75.0]|
|[13.2,15.9,49.6]|  5.6|[13.2,15.9,49.6]|
|[17.9,37.6,21.6]|  8.0|[17.9,37.6,21.6]|
|[18.7,12.1,23.4]|  6.7|[18.7,12.1,23.4]|
|[25.0,11.0,29.7]|  7.2|[25.0,11.0,29.7]|
|[25.1,25.7,43.3]|  8.5|[25.1,25.7,43.3]|
| [25.6,39.0,9.3]|  9.5| [25.6,39.0,9.3]|
| [27.5,1.6,20.7]|  6.9| [27.5,1.6,20.7]|
| [28.6,1.5,33.0]|  7.3| [28.6,1.5,33.0]|
| [39.5,41.1,5.8]| 10.8| [39.5,41.1,5.8]|
|[43.0,25.9,20.5]|  9.6|[43.0,25.9,20.5]|
|[44.5,39.3,45.1]| 10.4|[44.5,39.3,45.1]|
|[44.7,25.8,20.6]| 10.1|[44.7,25.8,20.6]|
|[50.0,11.6,18.4]|  8.4|[50.0,11.6,18.4]|
|[57.5,32.8,23.5]| 11.8|[57.5,32.8,23.5]|
|[59.6,12.0,43.1]|  9.7|[59.6,12.0,43.1]|
|[62.3,12.6,18.3]|  9.7|[62.3,12.6,18.3]|
+----------------+-----+----------

# 11. Evaluation

In [11]:
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 2.63325


In [12]:
y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))

r2_score: 0.7626545150141556
